In [1]:
import time
import pandas as pd
import streamlit as st
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.\n",
driver.get('https://www.redbus.in/')
time.sleep(5)
driver.maximize_window()
driver.execute_script("window.scrollBy(0, window.innerHeight)")
view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
view_all.click()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
kerala = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/ksrtc-kerala"]')
kerala.click()
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
kerala_route_names = []
kerala_route_links = []
time.sleep(3)

In [6]:
def scrape_data():
  route_names = driver.find_elements(By.CLASS_NAME, "route")
  for route_name in route_names:
      kerala_route_names.append(route_name.text)

  route_links = driver.find_elements(By.CLASS_NAME, "route")
  hrefs = [route_link.get_attribute('href') for route_link in route_links]
  for href in hrefs:
      kerala_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
for i in range(len(page_tabs)):
    if i > 0:
           page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")[i]
           driver.execute_script("arguments[0].click();", page_tabs) 
           time.sleep(4)
    scrape_data()
time.sleep(3)

df = pd.DataFrame(columns=['state_name' ,'route_name', 'route_link','bus_name', 'bus_type', 'departing', 
                           'duration', 'reaching', 'rating', 'price', 'availability'])
df_links= []
trial = []
seenlink = []
route_name = []
route_link = []
dict = {} 
count = 0
for link in kerala_route_links:
    driver.get(link)
    driver.maximize_window()
    if link not in seenlink:
        seenlink.append(link)
        try:
           wait = WebDriverWait(driver,3)
           wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
           wait_button = wait.until(EC.element_to_be_clickable((By.ID, "result-section" )))
           driver.execute_script("arguments[0].click();", wait_button)
        except NoSuchElementException:
           pass
        except TimeoutException:
           pass    
    
        for t in range(22):     #scrolling 22 times so that whole website loads
           driver.execute_script("window.scrollBy(0, 650);")
           time.sleep(0.5)
        names = driver.find_elements(By.CLASS_NAME,"travels")
        types = driver.find_elements(By.CLASS_NAME,"bus-type")
        depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
        travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
        arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
        star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
        fares = driver.find_elements(By.CLASS_NAME,"fare")
        seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")
        bus_name = []
        bus_type = []
        departing = []
        duration = []
        reaching = []
        rating = []
        price = []
        availability = []

        list(bus_name.append(i.text) for i in names)
        list(bus_type.append(i.text) for i in types)
        list(departing.append(i.text) for i in depart_timings)
        list(duration.append(i.text) for i in travel_durations)
        list(reaching.append(i.text) for i in arrivals)
        list(rating.append(i.text[:3]) for i in star_ratings)
        for fare in fares:
            if (fare.text).isdigit():
                price.append(fare.text)
            else:
                price.append((fare.text).replace('INR','').strip())
        list(availability.append(i.text) for i in seats_available)
        time.sleep(3)
        #remove duplicates on the names 
        print("range of bus_name for link:",range(len(bus_name)))
        print("link:",link)
        for i in range(len(bus_name)):
            dict={'state_name': "kerala",'route_name':kerala_route_names[count], 'route_link':link, 'bus_name':bus_name[i], 'bus_type':bus_type[i], 'departing':departing[i], 
                           'duration':duration[i], 'reaching':reaching[i], 'rating':rating[i], 'price':price[i], 'availability':availability[i]}
            df_links.append(dict)
    else:
         print("already seen")
    count = count +1
    

print("count =:",count)
#for i in range(len(df_links)):
   #print("value of i:",i)
   #print("Total Routes on each links",df_links[i])

df1=pd.DataFrame(df_links)
df=pd.concat([df,df1],ignore_index=True)

df.to_csv('kerala.csv', index=False)


range of bus_name for link: range(0, 24)
link: https://www.redbus.in/bus-tickets/bangalore-to-kozhikode
range of bus_name for link: range(0, 13)
link: https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam
range of bus_name for link: range(0, 25)
link: https://www.redbus.in/bus-tickets/kozhikode-to-bangalore
range of bus_name for link: range(0, 15)
link: https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode
range of bus_name for link: range(0, 13)
link: https://www.redbus.in/bus-tickets/kozhikode-to-mysore
range of bus_name for link: range(0, 5)
link: https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram
range of bus_name for link: range(0, 17)
link: https://www.redbus.in/bus-tickets/bangalore-to-kalpetta
range of bus_name for link: range(0, 10)
link: https://www.redbus.in/bus-tickets/mysore-to-kozhikode
range of bus_name for link: range(0, 22)
link: https://www.redbus.in/bus-tickets/kalpetta-to-bangalore
range of bus_name for link: range(0, 12)
link: https://www.redbu

In [7]:
print(df)

    state_name                      route_name  \
0       kerala          Bangalore to Kozhikode   
1       kerala          Bangalore to Kozhikode   
2       kerala          Bangalore to Kozhikode   
3       kerala          Bangalore to Kozhikode   
4       kerala          Bangalore to Kozhikode   
..         ...                             ...   
231     kerala  Kalpetta (kerala) to Kozhikode   
232     kerala  Kalpetta (kerala) to Kozhikode   
233     kerala  Kalpetta (kerala) to Kozhikode   
234     kerala  Kalpetta (kerala) to Kozhikode   
235     kerala  Kalpetta (kerala) to Kozhikode   

                                            route_link             bus_name  \
0    https://www.redbus.in/bus-tickets/bangalore-to...  Silver Line Travels   
1    https://www.redbus.in/bus-tickets/bangalore-to...          MMK Travels   
2    https://www.redbus.in/bus-tickets/bangalore-to...        Kyros Connect   
3    https://www.redbus.in/bus-tickets/bangalore-to...     AdSuraj Holidays   
4   

In [8]:
import pymysql


mydb= pymysql.connect(
 host="localhost",
 user="root",
 password="",
)
mycursor = mydb.cursor()
mydb.commit()
query="CREATE DATABASE project_redbus"
mycursor.execute(query)

1

In [9]:
query="use project_redbus"
mycursor.execute(query)


query="""CREATE TABLE BUS_DATA (
    id INT AUTO_INCREMENT PRIMARY KEY,
    state_name VARCHAR(70),
    route_name VARCHAR(70),
    route_link VARCHAR(180),
    bus_name VARCHAR(100),
    bus_type VARCHAR(50),
    departing VARCHAR(20),
    duration VARCHAR(20),
    reaching VARCHAR(20),
    rating FLOAT(2, 1),
    price DECIMAL(6, 2),
    availability INT
);"""
mycursor.execute(query)

0

In [15]:


query = """INSERT INTO BUS_DATA (state_name,route_name, route_link, bus_name, bus_type, departing, duration, reaching, rating, price, availability)
           VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for row in df_links:
    data = (row['state_name'],row['route_name'], row['route_link'], row['bus_name'], row['bus_type'], row['departing'], row['duration'], row['reaching'], row['rating'], row['price'], row['availability'])
    mycursor.execute(query,data)
    mydb.commit()


Andhra(APSRTC)

In [16]:
import time
import pandas as pd
import streamlit as st
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.\n",
driver.get('https://www.redbus.in/')
time.sleep(5)
driver.maximize_window()
driver.execute_script("window.scrollBy(0, 650)")
view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
view_all.click()
driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")
driver.maximize_window()
#APSRTC = driver.find_element(By.CSS_SELECTOR, "e")
#APSRTC.click()
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
andhra_route_names = []
andhra_route_links = []

def scrape_data():
  route_names = driver.find_elements(By.CLASS_NAME, "route")
  for route_name in route_names:
      andhra_route_names.append(route_name.text)

  route_links = driver.find_elements(By.CLASS_NAME, "route")
  hrefs = [route_link.get_attribute('href') for route_link in route_links]
  for href in hrefs:
      andhra_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
for i in range(len(page_tabs)):
    if i > 0:
           page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")[i]
           driver.execute_script("arguments[0].click();", page_tabs) 
           time.sleep(4)
    scrape_data()
print(andhra_route_names)

['Vijayawada to Hyderabad', 'Hyderabad to Vijayawada', 'Kakinada to Visakhapatnam', 'Visakhapatnam to Kakinada', 'Chittoor (Andhra Pradesh) to Bangalore', 'Kadapa to Bangalore', 'Anantapur (andhra pradesh) to Bangalore', 'Tirupati to Bangalore', 'Visakhapatnam to Vijayawada', 'Ongole to Hyderabad', 'Bangalore to Tirupati', 'Macherla (andhra pradesh) to Hyderabad', 'Rajahmundry to Visakhapatnam', 'Nandyal to Hyderabad', 'Bangalore to Kadapa', 'Hyderabad to Ongole', 'Guntur (Andhra Pradesh) to Hyderabad', 'Vijayawada to Visakhapatnam', 'Rajahmundry to Vijayawada', 'Hyderabad to Kurnool', 'Bangalore to Chittoor (Andhra Pradesh)', 'Kurnool to Hyderabad', 'Visakhapatnam to Rajahmundry', 'Rayachoti to Bangalore', 'Vinukonda to Hyderabad', 'Bangalore to Anantapur (andhra pradesh)', 'Amalapuram to Visakhapatnam', 'Vijayawada to Tirupati', 'Madanapalli to Bangalore', 'Narasaraopet to Hyderabad', 'Eluru to Hyderabad', 'Visakhapatnam to Amalapuram', 'Kadiri to Bangalore', 'Chennai to Tirupati', '

In [17]:

df1 = pd.DataFrame()
df_links= []
trial = []
seenlink = []
route_name = []
route_link = []
dict = {} 
count = 0
for link in andhra_route_links:
    driver.get(link)
    driver.maximize_window()
    if link not in seenlink:
        seenlink.append(link)
        try:
           wait = WebDriverWait(driver,3)
           wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
           wait_button = wait.until(EC.element_to_be_clickable((By.ID, "result-section" )))
           driver.execute_script("arguments[0].click();", wait_button)
        except NoSuchElementException:
           pass
        except TimeoutException:
           pass    
    
        for t in range(22):     #scrolling 22 times so that whole website loads
           driver.execute_script("window.scrollBy(0, 650);")
           time.sleep(0.5)
        names = driver.find_elements(By.CLASS_NAME,"travels")
        types = driver.find_elements(By.CLASS_NAME,"bus-type")
        depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
        travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
        arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
        star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
        fares = driver.find_elements(By.CLASS_NAME,"fare")
        seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")
        bus_name = []
        bus_type = []
        departing = []
        duration = []
        reaching = []
        rating = []
        price = []
        availability = []

        list(bus_name.append(i.text) for i in names)
        list(bus_type.append(i.text) for i in types)
        list(departing.append(i.text) for i in depart_timings)
        list(duration.append(i.text) for i in travel_durations)
        list(reaching.append(i.text) for i in arrivals)
        list(rating.append(i.text[:3]) for i in star_ratings)
        for fare in fares:
            if (fare.text).isdigit():
                price.append(fare.text)
            else:
                price.append((fare.text).replace('INR','').strip())
        list(availability.append(i.text) for i in seats_available)

        #remove duplicates on the names 
        print("range of bus_name for link:",range(len(bus_name)))
        print("link:",link)
        for i in range(len(bus_name)):
            dict={'state_name':"Andhra(APSRTC)",'route_name':andhra_route_names[count], 'route_link':link, 'bus_name':bus_name[i], 'bus_type':bus_type[i], 'departing':departing[i], 
                           'duration':duration[i], 'reaching':reaching[i], 'rating':rating[i], 'price':price[i], 'availability':availability[i]}
            df_links.append(dict)
    else:
         print("already seen")
    count = count +1

print("count =:",count)
for i in range(len(df_links)):
   print("value of i:",i)
   print("Total Routes on each links",df_links[i])

df1=pd.DataFrame(df_links)
df=pd.concat([df,df1],ignore_index=True)

df.to_csv('df_andhra.csv', index=False)
print(df)

range of bus_name for link: range(0, 60)
link: https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad
range of bus_name for link: range(0, 55)
link: https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada
range of bus_name for link: range(0, 11)
link: https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam
range of bus_name for link: range(0, 4)
link: https://www.redbus.in/bus-tickets/visakhapatnam-to-kakinada
range of bus_name for link: range(0, 7)
link: https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore
range of bus_name for link: range(0, 22)
link: https://www.redbus.in/bus-tickets/kadapa-to-bangalore
range of bus_name for link: range(0, 13)
link: https://www.redbus.in/bus-tickets/ananthapur-to-bangalore
range of bus_name for link: range(0, 57)
link: https://www.redbus.in/bus-tickets/tirupathi-to-bangalore
range of bus_name for link: range(0, 60)
link: https://www.redbus.in/bus-tickets/visakhapatnam-to-vijayawada
range of bus_name for link: range(0, 54)


In [18]:
import pymysql


mydb= pymysql.connect(
 host="localhost",
 user="root",
 password="",
)
mycursor = mydb.cursor()
mydb.commit()
query="use project_redbus"
mycursor.execute(query)
query = """INSERT INTO BUS_DATA (state_name,route_name, route_link, bus_name, bus_type, departing, duration, reaching, rating, price, availability)
           VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for row in df_links:
    data = (row['state_name'],row['route_name'], row['route_link'], row['bus_name'], row['bus_type'], row['departing'], row['duration'], row['reaching'], row['rating'], row['price'], row['availability'])
    mycursor.execute(query, data)
    mydb.commit()

Kadamba

In [19]:
import time
import pandas as pd
import streamlit as st
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.\n",
driver.get('https://www.redbus.in/')
time.sleep(5)
driver.maximize_window()
driver.execute_script("window.scrollBy(0, 650)")
view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
view_all.click()
driver.get("https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile")
driver.maximize_window()
#ktcl = driver.find_element(By.CLASS_NAME, 'a[href="/online-booking/ktcl"]')
#ktcl.click()
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
kadamba_route_names = []
kadamba_route_links = []

In [20]:

def scrape_data():
  route_names = driver.find_elements(By.CLASS_NAME, "route")
  for route_name in route_names:
      kadamba_route_names.append(route_name.text)

  route_links = driver.find_elements(By.CLASS_NAME, "route")
  hrefs = [route_link.get_attribute('href') for route_link in route_links]
  for href in hrefs:
      kadamba_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
for i in range(len(page_tabs)):
    if i > 0:
           page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")[i]
           driver.execute_script("arguments[0].click();", page_tabs) 
           time.sleep(4)
    scrape_data()



df_links= []
trial = []
seenlink = []
route_name = []
route_link = []
dict = {}
df1=pd.DataFrame() 
count = 0
for link in kadamba_route_links:
    driver.get(link)
    driver.maximize_window()
    if link not in seenlink:
        seenlink.append(link)
        try:
           wait = WebDriverWait(driver,3)
           wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
           wait_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))
           driver.execute_script("arguments[0].click();", wait_button)
        except NoSuchElementException:
           pass
        except TimeoutException:
           pass    
    
        for t in range(22):     #scrolling 22 times so that whole website loads
           driver.execute_script("window.scrollBy(0, 650);")
           time.sleep(0.5)
        names = driver.find_elements(By.CLASS_NAME,"travels")
        types = driver.find_elements(By.CLASS_NAME,"bus-type")
        depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
        travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
        arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
        star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
        fares = driver.find_elements(By.CLASS_NAME,"fare")
        seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")
        bus_name = []
        bus_type = []
        departing = []
        duration = []
        reaching = []
        rating = []
        price = []
        availability = []

        list(bus_name.append(i.text) for i in names)
        list(bus_type.append(i.text) for i in types)
        list(departing.append(i.text) for i in depart_timings)
        list(duration.append(i.text) for i in travel_durations)
        list(reaching.append(i.text) for i in arrivals)
        list(rating.append(i.text[:3]) for i in star_ratings)
        for fare in fares:
            if (fare.text).isdigit():
                price.append(fare.text)
            else:
                price.append((fare.text).replace('INR','').strip())
        list(availability.append(i.text) for i in seats_available)
        time.sleep(3)
        #remove duplicates on the names 
        print("range of bus_name for link:",range(len(bus_name)))
        print("link:",link)
        for i in range(len(bus_name)):
            dict={'state_name':"Kadamba(KTCL)",'route_name':kadamba_route_names[count], 'route_link':link, 'bus_name':bus_name[i], 'bus_type':bus_type[i], 'departing':departing[i], 
                           'duration':duration[i], 'reaching':reaching[i], 'rating':rating[i], 'price':price[i], 'availability':availability[i]}
            df_links.append(dict)
    else:
         print("already seen")
    count = count +1

print("count =:",count)
for i in range(len(df_links)):
   print("value of i:",i)
   print("Total Routes on each links",df_links[i])

df1=pd.DataFrame(df_links)
df=pd.concat([df,df1],ignore_index=True)

df.to_csv('Kadamba.csv', index=False)
print(df)

range of bus_name for link: range(0, 10)
link: https://www.redbus.in/bus-tickets/pune-to-goa
range of bus_name for link: range(0, 5)
link: https://www.redbus.in/bus-tickets/goa-to-pune
range of bus_name for link: range(0, 10)
link: https://www.redbus.in/bus-tickets/mumbai-to-goa
range of bus_name for link: range(0, 10)
link: https://www.redbus.in/bus-tickets/goa-to-mumbai
range of bus_name for link: range(0, 0)
link: https://www.redbus.in/bus-tickets/pandharpur-to-goa
range of bus_name for link: range(0, 9)
link: https://www.redbus.in/bus-tickets/bangalore-to-goa
range of bus_name for link: range(0, 2)
link: https://www.redbus.in/bus-tickets/goa-to-pandharpur
range of bus_name for link: range(0, 0)
link: https://www.redbus.in/bus-tickets/belagavi-to-goa
range of bus_name for link: range(0, 10)
link: https://www.redbus.in/bus-tickets/goa-to-bangalore
range of bus_name for link: range(0, 1)
link: https://www.redbus.in/bus-tickets/solapur-to-goa
range of bus_name for link: range(0, 10)
li

In [21]:
import pymysql


mydb= pymysql.connect(
 host="localhost",
 user="root",
 password="",
)
mycursor = mydb.cursor()
mydb.commit()
query="use project_redbus"
mycursor.execute(query)
query = """INSERT INTO BUS_DATA (state_name,route_name, route_link, bus_name, bus_type, departing, duration, reaching, rating, price, availability)
           VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for row in df_links:
    data = (row['state_name'],row['route_name'], row['route_link'], row['bus_name'], row['bus_type'], row['departing'], row['duration'], row['reaching'], row['rating'], row['price'], row['availability'])
    mycursor.execute(query, data)
    mydb.commit()

Bihar

In [22]:
import time
import pandas as pd
import streamlit as st
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.\n",
driver.get('https://www.redbus.in/')
time.sleep(5)
driver.maximize_window()
driver.execute_script("window.scrollBy(0, 650)")
view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
view_all.click()
driver.get("https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile")
driver.maximize_window()
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
bihar_route_names = []
bihar_route_links = []

In [23]:

def scrape_data():
  route_names = driver.find_elements(By.CLASS_NAME, "route")
  for route_name in route_names:
      bihar_route_names.append(route_name.text)

  route_links = driver.find_elements(By.CLASS_NAME, "route")
  hrefs = [route_link.get_attribute('href') for route_link in route_links]
  for href in hrefs:
      bihar_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
for i in range(len(page_tabs)):
    if i > 0:
           page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")[i]
           driver.execute_script("arguments[0].click();", page_tabs) 
           time.sleep(4)
    scrape_data()

df1=pd.DataFrame()
df_links= []
trial = []
seenlink = []
route_name = []
route_link = []
dict = {} 
count = 0
for link in bihar_route_links:
    driver.get(link)
    driver.maximize_window()
    if link not in seenlink:
        seenlink.append(link)
        try:
           wait = WebDriverWait(driver,3)
           wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
           wait_button = wait.until(EC.element_to_be_clickable((By.ID, "result-section" )))
           driver.execute_script("arguments[0].click();", wait_button)
        except NoSuchElementException:
           pass
        except TimeoutException:
           pass    
    
        for t in range(22):     #scrolling 22 times so that whole website loads
           driver.execute_script("window.scrollBy(0, 650);")
           time.sleep(0.5)
        names = driver.find_elements(By.CLASS_NAME,"travels")
        types = driver.find_elements(By.CLASS_NAME,"bus-type")
        depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
        travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
        arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
        star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
        fares = driver.find_elements(By.CLASS_NAME,"fare")
        seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")
        bus_name = []
        bus_type = []
        departing = []
        duration = []
        reaching = []
        rating = []
        price = []
        availability = []

        list(bus_name.append(i.text) for i in names)
        list(bus_type.append(i.text) for i in types)
        list(departing.append(i.text) for i in depart_timings)
        list(duration.append(i.text) for i in travel_durations)
        list(reaching.append(i.text) for i in arrivals)
        list(rating.append(i.text[:3]) for i in star_ratings)
        for fare in fares:
            if (fare.text).isdigit():
                price.append(fare.text)
            else:
                price.append((fare.text).replace('INR','').strip())
        list(availability.append(i.text) for i in seats_available)
        time.sleep(3)
        #remove duplicates on the names 
        #print("range of bus_name for link:",range(len(bus_name)))
        #print("link:",link)
        for i in range(len(bus_name)):
            dict={'state_name':"Bihar(BSRTC)",'route_name':bihar_route_names[count], 'route_link':link, 'bus_name':bus_name[i], 'bus_type':bus_type[i], 'departing':departing[i], 
                           'duration':duration[i], 'reaching':reaching[i], 'rating':rating[i], 'price':price[i], 'availability':availability[i]}
            df_links.append(dict)
    else:
         print("already seen")
    count = count +1

print("count =:",count)
for i in range(len(df_links)):
   print("value of i:",i)
   print("Total Routes on each links",df_links[i])

df1=pd.DataFrame(df_links)
df=pd.concat([df,df1],ignore_index=True)

df.to_csv('Bihar.csv', index=False)
print(df)

count =: 36
value of i: 0
Total Routes on each links {'state_name': 'Bihar(BSRTC)', 'route_name': 'Patna (Bihar) to Bettiah', 'route_link': 'https://www.redbus.in/bus-tickets/patna-to-bettiah', 'bus_name': 'Jai Mata Di Tour and Travels', 'bus_type': 'A/C Seater / Sleeper (2+2)', 'departing': '20:28', 'duration': '05h 32m', 'reaching': '02:00', 'rating': '3.4', 'price': '450', 'availability': '6 Seats available'}
value of i: 1
Total Routes on each links {'state_name': 'Bihar(BSRTC)', 'route_name': 'Patna (Bihar) to Bettiah', 'route_link': 'https://www.redbus.in/bus-tickets/patna-to-bettiah', 'bus_name': 'V.I.P Travels', 'bus_type': 'A/C Seater / Sleeper (2+2)', 'departing': '21:40', 'duration': '06h 00m', 'reaching': '03:40', 'rating': '3.0', 'price': '350', 'availability': '61 Seats available'}
value of i: 2
Total Routes on each links {'state_name': 'Bihar(BSRTC)', 'route_name': 'Patna (Bihar) to Bettiah', 'route_link': 'https://www.redbus.in/bus-tickets/patna-to-bettiah', 'bus_name': 

In [24]:
import pymysql


mydb= pymysql.connect(
 host="localhost",
 user="root",
 password="",
)
mycursor = mydb.cursor()
mydb.commit()
query="use project_redbus"
mycursor.execute(query)
query = """INSERT INTO BUS_DATA (state_name,route_name, route_link, bus_name, bus_type, departing, duration, reaching, rating, price, availability)
           VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for row in df_links:
    data = (row['state_name'],row['route_name'], row['route_link'], row['bus_name'], row['bus_type'], row['departing'], row['duration'], row['reaching'], row['rating'], row['price'], row['availability'])
    mycursor.execute(query, data)
    mydb.commit()

Rajasthan(RSRTC)

In [25]:
import time
import pandas as pd
import streamlit as st
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.\n",
driver.get('https://www.redbus.in/')
time.sleep(5)
driver.maximize_window()
driver.execute_script("window.scrollBy(0, 650)")
view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
view_all.click()
driver.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile")
driver.maximize_window()
#RSRTC = driver.find_element(By.CLASS_NAME, 'a[href="/online-booking/rsrtc"]')
#RSRTC.click()
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
rajasthan_route_names = []
rajasthan_route_links = []



In [26]:
def scrape_data():
  route_names = driver.find_elements(By.CLASS_NAME, "route")
  for route_name in route_names:
      rajasthan_route_names.append(route_name.text)

  route_links = driver.find_elements(By.CLASS_NAME, "route")
  hrefs = [route_link.get_attribute('href') for route_link in route_links]
  for href in hrefs:
      rajasthan_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
for i in range(len(page_tabs)):
    if i > 0:
           page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")[i]
           driver.execute_script("arguments[0].click();", page_tabs) 
           time.sleep(4)
    scrape_data()

    
df1=pd.DataFrame()
df_links= []
trial = []
seenlink = []
route_name = []
route_link = []
dict = {} 
count = 0
for link in rajasthan_route_links:
    driver.get(link)
    driver.maximize_window()
    if link not in seenlink:
        seenlink.append(link)
        try:
           wait = WebDriverWait(driver,3)
           wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
           wait_button = wait.until(EC.element_to_be_clickable((By.ID, "result-section")))
           driver.execute_script("arguments[0].click();", wait_button)
           
        except NoSuchElementException:
           pass
        except TimeoutException:
           pass    
    
        for t in range(22):     #scrolling 22 times so that whole website loads
           driver.execute_script("window.scrollBy(0, 650);")
           time.sleep(0.5)
        names = driver.find_elements(By.CLASS_NAME,"travels")
        types = driver.find_elements(By.CLASS_NAME,"bus-type")
        depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
        travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
        arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
        star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
        fares = driver.find_elements(By.CLASS_NAME,"fare")
        seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")
        bus_name = []
        bus_type = []
        departing = []
        duration = []
        reaching = []
        rating = []
        price = []
        availability = []

        list(bus_name.append(i.text) for i in names)
        list(bus_type.append(i.text) for i in types)
        list(departing.append(i.text) for i in depart_timings)
        list(duration.append(i.text) for i in travel_durations)
        list(reaching.append(i.text) for i in arrivals)
        list(rating.append(i.text[:3]) for i in star_ratings)
        for fare in fares:
            if (fare.text).isdigit():
                price.append(fare.text)
            else:
                price.append((fare.text).replace('INR','').strip())
        list(availability.append(i.text) for i in seats_available)
        time.sleep(3)
        #remove duplicates on the names 
        #print("range of bus_name for link:",range(len(bus_name)))
        #print("link:",link)
        for i in range(len(bus_name)):
            dict={'state_name':"Rajasthan(RSRTC)",'route_name':rajasthan_route_names[count], 'route_link':link, 'bus_name':bus_name[i], 'bus_type':bus_type[i], 'departing':departing[i], 
                           'duration':duration[i], 'reaching':reaching[i], 'rating':rating[i], 'price':price[i], 'availability':availability[i]}
            df_links.append(dict)
    else:
         print("already seen")
    count = count +1

print("count =:",count)
for i in range(len(df_links)):
   print("value of i:",i)
   print("Total Routes on each links",df_links[i])

df1=pd.DataFrame(df_links)
df=pd.concat([df,df1],ignore_index=True)

df.to_csv('Rajasthan.csv', index=False)
print(df)

count =: 21
value of i: 0
Total Routes on each links {'state_name': 'Rajasthan(RSRTC)', 'route_name': 'Jodhpur to Ajmer', 'route_link': 'https://www.redbus.in/bus-tickets/jodhpur-to-ajmer', 'bus_name': 'Jain travels regd', 'bus_type': 'NON AC Seater / Sleeper 2+1', 'departing': '19:40', 'duration': '04h 10m', 'reaching': '23:50', 'rating': '3.1', 'price': '330', 'availability': '18 Seats available'}
value of i: 1
Total Routes on each links {'state_name': 'Rajasthan(RSRTC)', 'route_name': 'Jodhpur to Ajmer', 'route_link': 'https://www.redbus.in/bus-tickets/jodhpur-to-ajmer', 'bus_name': 'FlixBus', 'bus_type': 'A/C Sleeper (2+1)', 'departing': '19:45', 'duration': '04h 05m', 'reaching': '23:50', 'rating': '5.0', 'price': '198', 'availability': '12 Seats available'}
value of i: 2
Total Routes on each links {'state_name': 'Rajasthan(RSRTC)', 'route_name': 'Jodhpur to Ajmer', 'route_link': 'https://www.redbus.in/bus-tickets/jodhpur-to-ajmer', 'bus_name': 'Chanakya Travels Agency', 'bus_type

In [27]:
import pymysql


mydb= pymysql.connect(
 host="localhost",
 user="root",
 password="",
)
mycursor = mydb.cursor()
mydb.commit()
query="use project_redbus"
mycursor.execute(query)
query = """INSERT INTO BUS_DATA (state_name,route_name, route_link, bus_name, bus_type, departing, duration, reaching, rating, price, availability)
           VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for row in df_links:
    data = (row['state_name'],row['route_name'], row['route_link'], row['bus_name'], row['bus_type'], row['departing'], row['duration'], row['reaching'], row['rating'], row['price'], row['availability'])
    mycursor.execute(query, data)
    mydb.commit()

southbengal

In [28]:
import time
import pandas as pd
import streamlit as st
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.\n",
driver.get('https://www.redbus.in/')
time.sleep(5)
driver.maximize_window()
driver.execute_script("window.scrollBy(0, 650)")
view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
view_all.click()
driver.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile")
driver.maximize_window()
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
southbengal_route_names = []
southbengal_route_links = []


print(southbengal_route_names)


[]


In [29]:

def scrape_data():
  route_names = driver.find_elements(By.CLASS_NAME, "route")
  for route_name in route_names:
      southbengal_route_names.append(route_name.text)

  route_links = driver.find_elements(By.CLASS_NAME, "route")
  hrefs = [route_link.get_attribute('href') for route_link in route_links]
  for href in hrefs:
      southbengal_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
for i in range(len(page_tabs)):
    if i > 0:
           page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")[i]
           driver.execute_script("arguments[0].click();", page_tabs) 
           time.sleep(4)
    scrape_data()



df1=pd.DataFrame()
df_links= []
trial = []
seenlink = []
route_name = []
route_link = []
dict = {} 
count = 0
for link in southbengal_route_links:
    driver.get(link)
    driver.maximize_window()
    if link not in seenlink:
        seenlink.append(link)
        try:
           wait = WebDriverWait(driver,3)
           wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
           wait_button = wait.until(EC.element_to_be_clickable((By.ID, "result-section")))
           driver.execute_script("arguments[0].click();", wait_button)
        except NoSuchElementException:
           pass
        except TimeoutException:
           pass    
    
        for t in range(22):     #scrolling 22 times so that whole website loads
           driver.execute_script("window.scrollBy(0, 650);")
           time.sleep(0.5)
        names = driver.find_elements(By.CLASS_NAME,"travels")
        types = driver.find_elements(By.CLASS_NAME,"bus-type")
        depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
        travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
        arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
        star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
        fares = driver.find_elements(By.CLASS_NAME,"fare")
        seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")
        bus_name = []
        bus_type = []
        departing = []
        duration = []
        reaching = []
        rating = []
        price = []
        availability = []

        list(bus_name.append(i.text) for i in names)
        list(bus_type.append(i.text) for i in types)
        list(departing.append(i.text) for i in depart_timings)
        list(duration.append(i.text) for i in travel_durations)
        list(reaching.append(i.text) for i in arrivals)
        list(rating.append(i.text[:3]) for i in star_ratings)
        for fare in fares:
            if (fare.text).isdigit():
                price.append(fare.text)
            else:
                price.append((fare.text).replace('INR','').strip())
        list(availability.append(i.text) for i in seats_available)
        time.sleep(3)
        #remove duplicates on the names 
        print("range of bus_name for link:",range(len(bus_name)))
        print("link:",link)
        for i in range(len(bus_name)):
            dict={'state_name':"Southbengal(SBSTC)",'route_name':southbengal_route_names[count], 'route_link':link, 'bus_name':bus_name[i], 'bus_type':bus_type[i], 'departing':departing[i], 
                           'duration':duration[i], 'reaching':reaching[i], 'rating':rating[i], 'price':price[i], 'availability':availability[i]}
            df_links.append(dict)
    else:
         print("already seen")
    count = count +1

print("count =:",count)
#for i in range(len(df_links)):
   #print("value of i:",i)
   #print("Total Routes on each links",df_links[i])

df1=pd.DataFrame(df_links)
df=pd.concat([df,df1],ignore_index=True)

df.to_csv('Southbengal.csv', index=False)
print(df)

range of bus_name for link: range(0, 0)
link: https://www.redbus.in/bus-tickets/burdwan-to-kolkata
range of bus_name for link: range(0, 4)
link: https://www.redbus.in/bus-tickets/kolkata-to-burdwan
range of bus_name for link: range(0, 0)
link: https://www.redbus.in/bus-tickets/durgapur-to-kolkata
range of bus_name for link: range(0, 0)
link: https://www.redbus.in/bus-tickets/kolkata-to-haldia
range of bus_name for link: range(0, 0)
link: https://www.redbus.in/bus-tickets/haldia-to-kolkata
range of bus_name for link: range(0, 1)
link: https://www.redbus.in/bus-tickets/kolkata-to-durgapur
range of bus_name for link: range(0, 0)
link: https://www.redbus.in/bus-tickets/kolkata-to-arambagh-west-bengal
range of bus_name for link: range(0, 5)
link: https://www.redbus.in/bus-tickets/digha-to-kolkata
range of bus_name for link: range(0, 5)
link: https://www.redbus.in/bus-tickets/kolkata-to-digha
range of bus_name for link: range(0, 0)
link: https://www.redbus.in/bus-tickets/kolkata-to-bankura
r

In [30]:
import pymysql


mydb= pymysql.connect(
 host="localhost",
 user="root",
 password="",
)
mycursor = mydb.cursor()
mydb.commit()
query="use project_redbus"
mycursor.execute(query)
query = """INSERT INTO BUS_DATA (state_name,route_name, route_link, bus_name, bus_type, departing, duration, reaching, rating, price, availability)
           VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for row in df_links:
    data = (row['state_name'],row['route_name'], row['route_link'], row['bus_name'], row['bus_type'], row['departing'], row['duration'], row['reaching'], row['rating'], row['price'], row['availability'])
    mycursor.execute(query, data)
    mydb.commit()

Himachal Pradesh

In [48]:
import time
import pandas as pd
import streamlit as st
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.\n",
driver.get('https://www.redbus.in/')
time.sleep(5)
driver.maximize_window()
driver.execute_script("window.scrollBy(0, 650)")
view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
view_all.click()
driver.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")
driver.maximize_window()
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
himachal_route_names = []
himachal_route_links = []


In [ ]:
def scrape_data():
  route_names = driver.find_elements(By.CLASS_NAME, "route")
  for route_name in route_names:
      himachal_route_names.append(route_name.text)

  route_links = driver.find_elements(By.CLASS_NAME, "route")
  hrefs = [route_link.get_attribute('href') for route_link in route_links]
  for href in hrefs:
      himachal_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
for i in range(len(page_tabs)):
    if i > 0:
           page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")[i]
           driver.execute_script("arguments[0].click();", page_tabs) 
           time.sleep(4)
    scrape_data()

df1=pd.DataFrame()
df_links= []
trial = []
seenlink = []
route_name = []
route_link = []
dict = {} 
count = 0
for link in himachal_route_links:
    driver.get(link)
    driver.maximize_window()

    if link not in seenlink:
        seenlink.append(link)
        try:
           wait = WebDriverWait(driver,3)
           wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
           wait_button = wait.until(EC.element_to_be_clickable((By.ID, "result-section")))
           driver.execute_script("arguments[0].click();", wait_button)
        except NoSuchElementException:
           pass
        except TimeoutException:
           pass    
    
        for t in range(22):     #scrolling 22 times so that whole website loads
           driver.execute_script("window.scrollBy(0, 650);")
           time.sleep(0.5)
        names = driver.find_elements(By.CLASS_NAME,"travels")
        types = driver.find_elements(By.CLASS_NAME,"bus-type")
        depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
        travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
        arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
        star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
        fares = driver.find_elements(By.CLASS_NAME,"fare")
        seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")
        bus_name = []
        bus_type = []
        departing = []
        duration = []
        reaching = []
        rating = []
        price = []
        availability = []

        list(bus_name.append(i.text) for i in names)
        list(bus_type.append(i.text) for i in types)
        list(departing.append(i.text) for i in depart_timings)
        list(duration.append(i.text) for i in travel_durations)
        list(reaching.append(i.text) for i in arrivals)
        list(rating.append(i.text[:3]) for i in star_ratings)
        for fare in fares:
            if (fare.text).isdigit():
                price.append(fare.text)
            else:
                price.append((fare.text).replace('INR','').strip())
        list(availability.append(i.text) for i in seats_available)
        time.sleep(3)
        #remove duplicates on the names 
        #print("range of bus_name for link:",range(len(bus_name)))
        #print("link:",link)
        for i in range(len(bus_name)):
            dict={'state_name':"Himachal(HPTDC)",'route_name':himachal_route_names[count], 'route_link':link, 'bus_name':bus_name[i], 'bus_type':bus_type[i], 'departing':departing[i], 
                           'duration':duration[i], 'reaching':reaching[i], 'rating':rating[i], 'price':price[i], 'availability':availability[i]}
            df_links.append(dict)
    else:
         print("already seen")
    count = count +1

#print("count =:",count)
#for i in range(len(df_links)):
   #print("value of i:",i)
   #print("Total Routes on each links",df_links[i])

df1=pd.DataFrame(df_links)
df=pd.concat([df,df1],ignore_index=True)

df.to_csv('Himachal.csv', index=False)
#print(df)

In [33]:
import pymysql


mydb= pymysql.connect(
 host="localhost",
 user="root",
 password="",
)
mycursor = mydb.cursor()
mydb.commit()
query="use project_redbus"
mycursor.execute(query)
query = """INSERT INTO BUS_DATA (state_name,route_name, route_link, bus_name, bus_type, departing, duration, reaching, rating, price, availability)
           VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for row in df_links:
    data = (row['state_name'],row['route_name'], row['route_link'], row['bus_name'], row['bus_type'], row['departing'], row['duration'], row['reaching'], row['rating'], row['price'], row['availability'])
    mycursor.execute(query, data)
    mydb.commit()

Assam

In [34]:

import time
import pandas as pd
import streamlit as st
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.\n",
driver.get('https://www.redbus.in/')
time.sleep(5)
driver.maximize_window()
driver.execute_script("window.scrollBy(0, 650)")
view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
view_all.click()
driver.get("https://www.redbus.in/online-booking/astc/?utm_source=rtchometile")
driver.maximize_window()
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
assam_route_names = []
assam_route_links = []



In [35]:
def scrape_data():
  route_names = driver.find_elements(By.CLASS_NAME, "route")
  for route_name in route_names:
      assam_route_names.append(route_name.text)

  route_links = driver.find_elements(By.CLASS_NAME, "route")
  hrefs = [route_link.get_attribute('href') for route_link in route_links]
  for href in hrefs:
      assam_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
for i in range(len(page_tabs)):
    if i > 0:
           page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")[i]
           driver.execute_script("arguments[0].click();", page_tabs) 
           time.sleep(4)
    scrape_data()

df1=pd.DataFrame()
df_links= []
trial = []
seenlink = []
route_name = []
route_link = []
dict = {} 
count = 0
for link in assam_route_links:
    driver.get(link)
    driver.maximize_window()
    if link not in seenlink:
        seenlink.append(link)
        try:
           wait = WebDriverWait(driver,3)
           wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
           wait_button = wait.until(EC.element_to_be_clickable((By.ID, "result-section")))
           driver.execute_script("arguments[0].click();", wait_button)
        except NoSuchElementException:
           pass
        except TimeoutException:
           pass    
    
        for t in range(22):     
           driver.execute_script("window.scrollBy(0, 650);")
           time.sleep(0.5)
        names = driver.find_elements(By.CLASS_NAME,"travels")
        types = driver.find_elements(By.CLASS_NAME,"bus-type")
        depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
        travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
        arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
        star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
        fares = driver.find_elements(By.CLASS_NAME,"fare")
        seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")
        bus_name = []
        bus_type = []
        departing = []
        duration = []
        reaching = []
        rating = []
        price = []
        availability = []

        list(bus_name.append(i.text) for i in names)
        list(bus_type.append(i.text) for i in types)
        list(departing.append(i.text) for i in depart_timings)
        list(duration.append(i.text) for i in travel_durations)
        list(reaching.append(i.text) for i in arrivals)
        list(rating.append(i.text[:3]) for i in star_ratings)
        for fare in fares:
            if (fare.text).isdigit():
                price.append(fare.text)
            else:
                price.append((fare.text).replace('INR','').strip())
        list(availability.append(i.text) for i in seats_available)
        time.sleep(3)
        #remove duplicates on the names 
        print("range of bus_name for link:",range(len(bus_name)))
        print("link:",link)
        for i in range(len(bus_name)):
            dict={'state_name':"Assam(ASTC)",'route_name':assam_route_names[count], 'route_link':link, 'bus_name':bus_name[i], 'bus_type':bus_type[i], 'departing':departing[i], 
                           'duration':duration[i], 'reaching':reaching[i], 'rating':rating[i], 'price':price[i], 'availability':availability[i]}
            df_links.append(dict)
    else:
         print("already seen")
    count = count +1

print("count =:",count)
for i in range(len(df_links)):
   print("value of i:",i)
   print("Total Routes on each links",df_links[i])

df1=pd.DataFrame(df_links)
df=pd.concat([df,df1],ignore_index=True)

df.to_csv('Assam.csv', index=False)
print(df)

range of bus_name for link: range(0, 10)
link: https://www.redbus.in/bus-tickets/tezpur-to-guwahati
range of bus_name for link: range(0, 5)
link: https://www.redbus.in/bus-tickets/guwahati-to-tezpur
range of bus_name for link: range(0, 10)
link: https://www.redbus.in/bus-tickets/nagaon-to-guwahati
range of bus_name for link: range(0, 5)
link: https://www.redbus.in/bus-tickets/guwahati-to-nagaon
range of bus_name for link: range(0, 5)
link: https://www.redbus.in/bus-tickets/goalpara-to-guwahati
range of bus_name for link: range(0, 3)
link: https://www.redbus.in/bus-tickets/jorhat-to-north-lakhimpur
range of bus_name for link: range(0, 5)
link: https://www.redbus.in/bus-tickets/dhubri-to-guwahati
range of bus_name for link: range(0, 5)
link: https://www.redbus.in/bus-tickets/guwahati-to-dhubri
range of bus_name for link: range(0, 1)
link: https://www.redbus.in/bus-tickets/north-lakhimpur-to-sibsagar
range of bus_name for link: range(0, 3)
link: https://www.redbus.in/bus-tickets/north-lak

In [36]:
import pymysql


mydb= pymysql.connect(
 host="localhost",
 user="root",
 password="",
)
mycursor = mydb.cursor()
mydb.commit()
query="use project_redbus"
mycursor.execute(query)
query = """INSERT INTO BUS_DATA (state_name,route_name, route_link, bus_name, bus_type, departing, duration, reaching, rating, price, availability)
           VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for row in df_links:
    data = (row['state_name'],row['route_name'], row['route_link'], row['bus_name'], row['bus_type'], row['departing'], row['duration'], row['reaching'], row['rating'], row['price'], row['availability'])
    mycursor.execute(query, data)
    mydb.commit()

Chandigarh

In [40]:

import time
import pandas as pd
import streamlit as st
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.\n",
driver.get('https://www.redbus.in/')
time.sleep(5)
driver.maximize_window()
driver.execute_script("window.scrollBy(0, 650)")
view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
view_all.click()
driver.get("https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu")
driver.maximize_window()
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
chandigarh_route_names = []
chandigarh_route_links = []


In [41]:
def scrape_data():
  route_names = driver.find_elements(By.CLASS_NAME, "route")
  for route_name in route_names:
      chandigarh_route_names.append(route_name.text)

  route_links = driver.find_elements(By.CLASS_NAME, "route")
  hrefs = [route_link.get_attribute('href') for route_link in route_links]
  for href in hrefs:
      chandigarh_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
for i in range(len(page_tabs)):
    if i > 0:
           page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")[i]
           driver.execute_script("arguments[0].click();", page_tabs) 
           time.sleep(4)
    scrape_data()

df1=pd.DataFrame()
df_links= []
trial = []
seenlink = []
route_name = []
route_link = []
dict = {} 
count = 0
for link in chandigarh_route_links:
    driver.get(link)
    driver.maximize_window()
    if link not in seenlink:
        seenlink.append(link)
        try:
           wait = WebDriverWait(driver,3)
           wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
           wait_button = wait.until(EC.element_to_be_clickable((By.ID, "result-section")))
           driver.execute_script("arguments[0].click();", wait_button)
        except NoSuchElementException:
           pass
        except TimeoutException:
           pass    
    
        for t in range(22):     #scrolling 22 times so that whole website loads
           driver.execute_script("window.scrollBy(0, 650);")
           time.sleep(0.5)
        names = driver.find_elements(By.CLASS_NAME,"travels")
        types = driver.find_elements(By.CLASS_NAME,"bus-type")
        depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
        travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
        arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
        star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
        fares = driver.find_elements(By.CLASS_NAME,"fare")
        seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")
        bus_name = []
        bus_type = []
        departing = []
        duration = []
        reaching = []
        rating = []
        price = []
        availability = []

        list(bus_name.append(i.text) for i in names)
        list(bus_type.append(i.text) for i in types)
        list(departing.append(i.text) for i in depart_timings)
        list(duration.append(i.text) for i in travel_durations)
        list(reaching.append(i.text) for i in arrivals)
        list(rating.append(i.text[:3]) for i in star_ratings)
        for fare in fares:
            if (fare.text).isdigit():
                price.append(fare.text)
            else:
                price.append((fare.text).replace('INR','').strip())
        list(availability.append(i.text) for i in seats_available)
        time.sleep(3)
        #remove duplicates on the names 
        print("range of bus_name for link:",range(len(bus_name)))
        print("link:",link)
        for i in range(len(bus_name)):
            dict={'state_name': "Chandigarh(CTU)",'route_name':chandigarh_route_names[count], 'route_link':link, 'bus_name':bus_name[i], 'bus_type':bus_type[i], 'departing':departing[i], 
                           'duration':duration[i], 'reaching':reaching[i], 'rating':rating[i], 'price':price[i], 'availability':availability[i]}
            df_links.append(dict)
    else:
         print("already seen")
    count = count +1

print("count =:",count)
for i in range(len(df_links)):
   print("value of i:",i)
   print("Total Routes on each links",df_links[i])

df1=pd.DataFrame(df_links)
df=pd.concat([df,df1],ignore_index=True)

df.to_csv('Chandigarh.csv', index=False)

range of bus_name for link: range(0, 60)
link: https://www.redbus.in/bus-tickets/chandigarh-to-delhi
range of bus_name for link: range(0, 65)
link: https://www.redbus.in/bus-tickets/delhi-to-chandigarh
range of bus_name for link: range(0, 7)
link: https://www.redbus.in/bus-tickets/yamuna-nagar-to-chandigarh
range of bus_name for link: range(0, 15)
link: https://www.redbus.in/bus-tickets/chandigarh-to-shimla
range of bus_name for link: range(0, 3)
link: https://www.redbus.in/bus-tickets/chandigarh-to-vrindavan
range of bus_name for link: range(0, 6)
link: https://www.redbus.in/bus-tickets/chandigarh-to-yamuna-nagar
range of bus_name for link: range(0, 2)
link: https://www.redbus.in/bus-tickets/chandigarh-to-sujanpur
range of bus_name for link: range(0, 37)
link: https://www.redbus.in/bus-tickets/ludhiana-to-chandigarh
range of bus_name for link: range(0, 3)
link: https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-chandigarh
range of bus_name for link: range(0, 3)
link: https

In [42]:
import pymysql


mydb= pymysql.connect(
 host="localhost",
 user="root",
 password="",
)
mycursor = mydb.cursor()
mydb.commit()
query="use project_redbus"
mycursor.execute(query)
query = """INSERT INTO BUS_DATA (state_name,route_name, route_link, bus_name, bus_type, departing, duration, reaching, rating, price, availability)
           VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for row in df_links:
    data = (row['state_name'],row['route_name'], row['route_link'], row['bus_name'], row['bus_type'], row['departing'], row['duration'], row['reaching'], row['rating'], row['price'], row['availability'])
    mycursor.execute(query, data)
    mydb.commit()

Uttarpradesh

In [45]:

import time
import pandas as pd
import streamlit as st
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome()  
driver.get('https://www.redbus.in/')
time.sleep(5)
driver.maximize_window()
driver.execute_script("window.scrollBy(0, 650)")
view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
view_all.click()
driver.get("https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile")
driver.maximize_window()
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
driver.execute_script("window.scrollBy(0, window.innerHeight)")
up_route_names = []
up_route_links = []


In [46]:


def scrape_data():
  route_names = driver.find_elements(By.CLASS_NAME, "route")
  for route_name in route_names:
      up_route_names.append(route_name.text)

  route_links = driver.find_elements(By.CLASS_NAME, "route")
  hrefs = [route_link.get_attribute('href') for route_link in route_links]
  for href in hrefs:
      up_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
for i in range(len(page_tabs)):
    if i > 0:
           page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")[i]
           driver.execute_script("arguments[0].click();", page_tabs) 
           time.sleep(4)
    scrape_data()
print(up_route_names)


df_links= []
trial = []
seenlink = []
route_name = []
route_link = []
dict = {} 
df=pd.DataFrame()
count = 0
for link in up_route_links:
    driver.get(link)
    driver.maximize_window()
    if link not in seenlink:
        seenlink.append(link)
        try:
           wait = WebDriverWait(driver,3)
           wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
           wait_button = wait.until(EC.element_to_be_clickable((By.ID, "result-section")))
           driver.execute_script("arguments[0].click();", wait_button)
        except NoSuchElementException:
           pass
        except TimeoutException:
           pass    
    
        for t in range(22):     #scrolling 22 times so that whole website loads
           driver.execute_script("window.scrollBy(0, 650);")
           time.sleep(0.5)
        names = driver.find_elements(By.CLASS_NAME,"travels")
        types = driver.find_elements(By.CLASS_NAME,"bus-type")
        depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
        travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
        arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
        star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
        fares = driver.find_elements(By.CLASS_NAME,"fare")
        seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")
        bus_name = []
        bus_type = []
        departing = []
        duration = []
        reaching = []
        rating = []
        price = []
        availability = []

        list(bus_name.append(i.text) for i in names)
        list(bus_type.append(i.text) for i in types)
        list(departing.append(i.text) for i in depart_timings)
        list(duration.append(i.text) for i in travel_durations)
        list(reaching.append(i.text) for i in arrivals)
        list(rating.append(i.text[:3]) for i in star_ratings)
        for fare in fares:
            if (fare.text).isdigit():
                price.append(fare.text)
            else:
                price.append((fare.text).replace('INR','').strip())
        list(availability.append(i.text) for i in seats_available)
        time.sleep(3)
        #remove duplicates on the names 
        #print("range of bus_name for link:",range(len(bus_name)))
        #print("link:",link)
        for i in range(len(bus_name)):
            dict={'state_name':"Uttarpradesh(UPSRTC)",'route_name':up_route_names[count], 'route_link':link, 'bus_name':bus_name[i], 'bus_type':bus_type[i], 'departing':departing[i], 
                           'duration':duration[i], 'reaching':reaching[i], 'rating':rating[i], 'price':price[i], 'availability':availability[i]}
            df_links.append(dict)
    else:
         print("already seen")
    count = count +1

print("count =:",count)
for i in range(len(df_links)):
   print("value of i:",i)
   print("Total Routes on each links",df_links[i])

df1=pd.DataFrame(df_links)
df=pd.concat([df,df1],ignore_index=True)

df.to_csv('Uttarpradesh.csv', index=False)

['Bareilly to Delhi', 'Aligarh (uttar pradesh) to Delhi', 'Delhi to Bareilly', 'Delhi to Aligarh (uttar pradesh)', 'Farrukhabad (Uttar Pradesh) to Delhi', 'Badaun to Delhi', 'Lucknow to Allahabad', 'Lucknow to Varanasi', 'Delhi to Badaun', 'Agra to Bareilly', 'Allahabad to Lucknow', 'Sitapur (Uttar Pradesh) to Delhi', 'Delhi to Farrukhabad (Uttar Pradesh)', 'Moradabad to Delhi', 'Delhi to Haridwar', 'Delhi to Sitapur (Uttar Pradesh)', 'Bareilly to Agra', 'Agra to Lucknow', 'Kanpur (Uttar Pradesh) to Jhansi', 'Lucknow to Agra', 'Varanasi to Lucknow', 'Bareilly to Haridwar', 'Delhi to Agra', 'Delhi to Shahjahanpur (Uttar Pradesh)', 'Agra to Delhi', 'Lucknow to Ballia', 'Bareilly to Dehradun', 'Allahabad to Azamgarh', 'Kanpur (Uttar Pradesh) to Bareilly', 'Lucknow to Ayodhya', 'Aligarh (uttar pradesh) to Lucknow', 'Aligarh (uttar pradesh) to Agra', 'Lucknow to Haridwar', 'Lucknow to Kanpur (Uttar Pradesh)', 'Lucknow to Aligarh (uttar pradesh)', 'Delhi to Etah (Uttar Pradesh)', 'Delhi to K

In [47]:
import pymysql


mydb= pymysql.connect(
 host="localhost",
 user="root",
 password="",
)
mycursor = mydb.cursor()
mydb.commit()
query="use project_redbus"
mycursor.execute(query)
query = """INSERT INTO BUS_DATA (state_name,route_name, route_link, bus_name, bus_type, departing, duration, reaching, rating, price, availability)
           VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for row in df_links:
    data = (row['state_name'],row['route_name'], row['route_link'], row['bus_name'], row['bus_type'], row['departing'], row['duration'], row['reaching'], row['rating'], row['price'], row['availability'])
    mycursor.execute(query, data)
    mydb.commit()